<a href="https://colab.research.google.com/github/alexstar55/stitching_heatmap/blob/master/DailyNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0712  




### 记事  


#### plan:

- 修改gene_maskGaus.py，生成单张与视频heatmap;  
- 写datastructure笔记，操作selenium eclipse
- fastai lec13

#### done

- 

### 小代码与Tips

#### [如何在 GitHub.com 上删除某个 Repository 中的某个文件夹？](https://www.zhihu.com/question/20418177)  


步骤：（以删除.idea文件夹为例）  
`git rm -r --cached .idea  #--cached不会把本地的.idea删除`  
`git commit -m 'delete .idea dir'`  
`git push -u origin master(或者git push -u origin dev)  
`

#### [tmux加鼠标滚屏](https://www.jianshu.com/p/c217318a9380)  
 tmux 提供了一些个性化配置项来优化这些配置，首先在 shell 中运行 touch ~/.tmux.conf 新建用户配置文件。在文件中增加以下内容：  

 

 ```
 # 开启鼠标模式
set -g mode-mouse on

# 允许鼠标选择窗格
set -g mouse-select-pane on

# 如果喜欢给窗口自定义命名，那么需要关闭窗口的自动命名
set-option -g allow-rename off

# 如果对 vim 比较熟悉，可以将 copy mode 的快捷键换成 vi 模式
set-window-option -g mode-keys vi
```

#### 获得文件修改时间，检查文件是否存在

本地时间：time.strftime('%Y%m%d', time.localtime(time.time()))  


- 涉及文件是否存在，与获得修改时间的代码片段
pana_name="pana_last_"+shop[0]+".jpg" #sth like "pana_last_S0027"
        if os.path.exists(os.path.join(bg_path,pana_name)): #if the shop's bg pic exists
            filemt = time.localtime(os.stat(
                os.path.join(bg_path, pana_name)).st_mtime)  # get the touch time of the bg file
            if (int(time.strftime("%Y%m%d", filemt))<date_now)

## 0715

### 记事

- 阈值代码修改
- 爬虫下载——>Undone
- lec13

## 0717

### 记事

- gene video调试、测试ok
- py36本地环境[成功解决 module 'cv2' has no attribute 'xfeatures2d'](https://blog.csdn.net/Sunny_Future/article/details/87949013)：需要卸载opencv，只安装opencv-contrib-python  
后证实是opencv要3.4.1.15，而3.3版本不行
- 下载视频代码与展示代码调试

## 0718

### 记事

- 模拟5小时数据测试处理ok
- max_thresh, inter_thresh两种阈值代码修改
- 下载视频代码与展示代码调试

## 0719

### 记事

- 网页模拟
- 图片展示改进

Path(savepth).mkdir(parents=True, exist_ok=True)可以新建好几层目录

## 0722

### 记事

- 网页模拟图片大小修改（尽量大），阈值修改 以显示红色
- 加代码使用潘执政yolo检测人
- 刘兵 协助改图片缩放  css使用
- 上传python学习文件


## 0723

### 记事

- 网页模拟图片字体修改
- 准备定时运行代码
- 翻译。188行，5月？
- fastai lec13, pandas处理kaggle
- 上传python学习文件


## 0724

### 记事

- 热力图色谱 自定义
- 阈值改变
- 网页标题调整。春哥翻译完成


In [0]:
def gene1heatpic(img_paths,rec_paths,starttime,endtime,threshold,a_list): #txtlst
    '''

    :param img_paths: no person pictures of 1 shop, sth like [PosixPath('picture/S0027_z100256_20190712.jpg'),
   PosixPath('picture/S0027_z100257_20190712.jpg'),   PosixPath('picture/S0027_z100258_20190712.jpg')])]
    :param rec_paths:  records of 1 shop, sth like [PosixPath('script/S0027_z100256_20190712r.txt'),
   PosixPath('script/S0027_z100257_20190712r.txt'),   PosixPath('script/S0027_z100258_20190712r.txt')]
    :param starttime:
    :param endtime:
    :param threshold:
    :param a_list:
    :return:name, panorama[-1].image
    '''
    imgs = list()
    for index, txt in enumerate(rec_paths):
        flp=time2flp(txt,starttime,endtime)
        #print(flp)
        a = person_add_up(txt, flp, temp=False)
        a[720:755, 362:397] = 0
        static = []
        static_1 = []
        static_2 = []
        print('firt_max', a.max())
        a_max = a.max()
        print('diedai', int(a_max))
        for i in range(1, int(a_max) + 1):
            num = len(np.where(a == i)[0])
            if num != 0:
                static_1.append(i)
                static.append(num)

                if a_max < threshold:
                    static_2.append(i)
                    continue
                if i > threshold:
                    mins = threshold
                    i_cha_norm = (2 * i - a_list[index] - mins) * v / (a_list[index] - mins)  # [17-59]缩放到[-v,v]
                    sig = sigmoid(i_cha_norm) #* 10  # [-v,v]压缩到(0,1)
                    a[a == i] = a[a == i] + mins - i + sig  # [17-59]变成[17.04,17.95]
                    static_2.append(i)
        a = a / (threshold + 1)
        print('a_max', a.max())
        toadd = a * 255
        print('toadd_max', toadd.max())
        toadd[toadd > 255] = 255
        image_heatmap2 = toadd.astype(np.uint8)
        image_heatmap3 = cv2.cvtColor(image_heatmap2, cv2.COLOR_GRAY2BGR)
        image_heatmap3 = cv2.GaussianBlur(image_heatmap3, (35, 35), 0)
        image_heatmap3 = cv2.cvtColor(image_heatmap3, cv2.COLOR_BGR2GRAY)
        #This is to reverse color
        img_info = image_heatmap3.shape
        image_height = img_info[0]
        image_weight = img_info[1]
        dst = np.zeros((image_height, image_weight, 1), np.uint8)
        for i in range(image_height):
            for j in range(image_weight):
                grayPixel = image_heatmap3[i][j]
                dst[i][j] = 255 - grayPixel
        image_heatmap3 = dst
        ##########################

        # im_gray = cv2.imread("pluto.jpg", cv2.IMREAD_GRAYSCALE)
        # im_color = cv2.applyColorMap(im_gray, cv2.COLORMAP_HSV)
        startcolor = '#ff0000'  # 红色，可以自行修改
        midcolor = '#80ff00'  # 绿色，可以自行修改
        endcolor = '#0080ff'  # 蓝色，可以自行修改
        cmap2 = col.LinearSegmentedColormap.from_list('own2', [startcolor, '#ffbf00','#ffff00', '#bfff00',midcolor, '#40ff00', '#00ff40', '#00ffbf',endcolor])
        # extra arguments are N=256, gamma=1.0
        cm.register_cmap(cmap=cmap2)

        image_heatmap4 = cv2.applyColorMap(image_heatmap3, cmapy.cmap('own2'))
        image_heatmap4[a == 0] = 0
        alpha = 0.18  # 这是调节透明度
        #openname = txt.split('.')[0]
        #openname = openname.rstrip('r')
        im19 = cv2.imread(str(img_paths[index]))  # 这是彩色无人背景图
        im19 = cv2.resize(im19, (int(img_width), int(img_height)))
        image2 = im19.copy()
        image_gray_mask = image_heatmap2
        image2[image_gray_mask > 0] = image_heatmap4[image_gray_mask > 0]
        imgs.append(cv2.addWeighted(im19, 1 - alpha, image2, alpha, 1))

    #image_names = rec_paths #['picture/1101.jpg', 'picture/1102.jpg', 'picture/1103.jpg']
    images = []
    panorama = []
    for i in range(0, len(img_paths)): #img is the same number with rec
        print("Loading " + str(i))
        # img = resize_image(img)
        images.append(Image(str(i), imgs[i]))
    panorama.append(Image(images[0].name, images[0].image))
    count0 = 0
    print("Your images have been loaded. Generating panorama starts ...")
    name = ('_').join([os.path.split(img_paths[0])[1].split('_')[0],str(endtime)]) #+'.jpg'
    shopn = name.split('_')[0]
    for i in range(0, len(images) - 1):
        panorama.append(Image(str(i + 1), make_panorama(panorama[i], images[i + 1], count0, shopn)))
        count0 += 1
    print("A panorama image is generated.")
    #name = './picture/results/pana' + str(flp) + '.jpg'
    #cv2.imwrite(os.path.join(RST_PATH,name), panorama[-1].image)
    return name, panorama[-1].image

## 0725

### 记事

- aria2下载
- 定时运行代码apschedule与k8s学习
- 页面图个数调整


## 0726

### 记事

- 定时运行代码apschedule与k8s学习
- 页面图个数调整
- 安装opencv-contrib-python3.4.1.15卸载opencv, conda install hdf5>1.10.3重安cmapy.24min 处理30min。使用.background import BackgroundScheduler


定时运行部分代码

```
executors = {
        'default':ThreadPoolExecutor(10),
    'processpool': ProcessPoolExecutor(5)
    }
    job_defaults = {      #coalescing合并参数就能把一个多次的哑弹任务揉成一个一次的哑弹任务
        'coalesce': True,  #If one job missed, then pass to the next time
        'max_instances': 3,
        'misfire_grace_time': 900
        #检查它预订运行的时间和当下时间的差值，大于我们设置的900秒限制，那么这个运行实例不会被执行
        } #'max_instances': 3


    scheduler = BackgroundScheduler(executors=executors, job_defaults=job_defaults)
    scheduler.add_job(run_main, 'cron', hour='1-23', minute='41')
    print('Press Ctrl+{0} to exit'.format('Break' if os.name == 'nt' else 'C    '))

    try:
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        pass
```

页面子图展示代码

In [0]:
def show_inpic(rstpath=HEATRST_PATH, savepth=TOSHOW_PATH):
    rstimgs = sorted(Path(rstpath).glob('*.jpg'))
    Path(savepth).mkdir(parents=True, exist_ok=True)
    h, w, _ = cv2.imread(str(rstimgs[0])).shape
    pic_1page = round(32 * h / (w * 9))*2 #all_h/2 * (1pic_w/1pic_h) * pics_of1row =all_w -->pics_of 1row*2 is pic_1page
    #pic_1page = 8
    rown = 2
    coln = pic_1page / rown
    pic_n = math.ceil(len(rstimgs) / pic_1page)  # 一共pic_n张大图，每图里有6张结果图
    for i in range(pic_n):
        fig = plt.figure(figsize=(25, 14))  # 循环各图
        plt.subplots_adjust(left=0.01, bottom=0.01, right=0.995, top=0.95, hspace=0.005, wspace=0.0)
        for j in range(pic_1page):
            ind = i * pic_1page + j  # get index of the picture
            if ind > (len(rstimgs) - 1):
                plt.savefig(savepth + str(i) + ".jpg")
                break
            indtm = ind % pic_1page
            r, c = indtm % rown, (indtm % pic_1page) // 2  # 获得matplotlib的行列位置
            p = r * coln + c + 1  # 获得matplotlib的序号位置
            ax = fig.add_subplot(rown, coln, p)
            image = cv2.imread(str(rstimgs[ind]))
            #         print(str(ind))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.xticks([]), plt.yticks([])
            if r == 0:  # 如果是第0行，加个小标题
                title0 = os.path.split(rstimgs[ind])[1].split('.')[0]
                title0 = title0[6:8] + "時" +title0[8:10]
                ax.set_title(title0, fontproperties=fonts, fontsize=28)
                if c == 0:
                    title1 = "時間帯別推移"
                    ax.set_ylabel(title1, fontproperties=fonts, fontsize=28)
            if r == 1 and c == 0:
                title1 = "時間帯別累積"
                ax.set_ylabel(title1, fontproperties=fonts, fontsize=28)
            plt.imshow(image)
            if p == pic_1page:
                print(i)
                plt.savefig(os.path.join(savepth , str(i) + ".jpg"))

## 0730

### 记事

- 数据结构排序算法小结
- fastai lec13
- kaggle
- docker file调查

## 0731

- k8s学习